In [1]:
# This notebook contains cluster analysis code used to idnetify natural groupinps that may 
#    exist in a set of features.  Specifically, we looked for groupings in the column 
#    containing earnings info (MN_EARN_WNE_MALE1_P10).
# Our aim was to use this grouping to bin column data for Decision Tree and PCA analyses.
# The goal was to improve model accuracy by using natural binning instead of conventional 
#    20K band grouping. We found that natural binning actually reduced accuracy; it is not 
#    clear why this would be the case.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [3]:
data = pd.read_csv('consol_data.csv')
print(data.shape)

(1060, 71)


In [4]:
edu_df = data.replace('PrivacySuppressed', np.nan).dropna()
print(edu_df.shape)

(989, 71)


In [5]:
edu_df.columns

Index(['UNITID', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'PCT_WHITE', 'PCT_BLACK',
       'PCT_ASIAN', 'PCT_HISPANIC', 'PCT_BORN_US', 'MEDIAN_HH_INC',
       'LN_MEDIAN_HH_INC', 'POVERTY_RATE', 'UNEMP_RATE', 'LATITUDE',
       'LONGITUDE', 'ADM_RATE', 'ADM_RATE_ALL', 'SAT_AVG', 'COUNT_NWNE_P10',
       'COUNT_WNE_P10', 'MN_EARN_WNE_P10', 'MN_EARN_WNE_MALE0_P10',
       'MN_EARN_WNE_MALE1_P10', 'TUITIONFEE_IN', 'C150_4_WHITE',
       'C150_4_BLACK', 'C150_4_HISP', 'C150_4_ASIAN', 'PCIP01', 'PCIP03',
       'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13',
       'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24',
       'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38',
       'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45',
       'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52',
       'PCIP54', 'AGE_ENTRY', 'FEMALE', 'FIRST_GEN', 'FAMINC'],
      dtype='object')

In [ ]:
subset_df = edu_df[['SAT_AVG', 'MEDIAN_HH_INC', 'TUITIONFEE_IN']].copy()
subset_df.head()

In [ ]:
sse = []
for k in range(1, 12):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(subset_df)
    sse.append(kmeans.inertia_)
plt.plot(range(1, 12), sse)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans = kmeans.fit(subset_df)
# Getting the cluster labels
predicted_clusters = kmeans.predict(subset_df)
# Centroid values
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
# The two code blocks below were used to identify natural groupings in earnings.
# The centroids found represent centers of natrual bands in earning, which were
#    used to drive Decision Tree modeling--results obtained w/ this binning
#    scheme produced lower accuracy.

In [ ]:
subset_df = edu_df[['MN_EARN_WNE_MALE0_P10']].copy()
sse = []
for k in range(1, 12):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(subset_df)
    sse.append(kmeans.inertia_)
plt.plot(range(1, 12), sse)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans = kmeans.fit(subset_df)
# Getting the cluster labels
predicted_clusters = kmeans.predict(subset_df)
# Centroid values
centroids = kmeans.cluster_centers_
print(centroids)

## Test code not used for the project
This code was used to visulaize any natural grouping that may exist between race (represented by % black student
population) and earning potential.

In [ ]:
y=subset_df.iloc[:, 1].values
X=subset_df.iloc[:, 0].values
plt.scatter(X, y, marker='.')
plt.show()

In [ ]:
plt.scatter(X, y, marker='.', c=predicted_clusters, s=50, cmap='viridis')
plt.show()

In [ ]:
# Repeat the process w/ minmax scaling applied to the dataframe

In [ ]:
X = subset_df[['PCT_BLACK']]
y = subset_df[['Mean earnings of students working and not enrolled 10 years after entry']]
X_minmax = MinMaxScaler().fit(X)
y_minmax = MinMaxScaler().fit(y)
X_transformed = X_minmax.transform(X)
y_tranformed = y_minmax.transform(y)
plt.scatter(X_transformed, y_tranformed, marker='.')
plt.show()

In [ ]:
scaler = preprocessing.MinMaxScaler()
minmax_scaled_df = scaler.fit_transform(subset_df)
minmax_scaled_df = pd.DataFrame(minmax_scaled_df, columns=['PCT', 'Earning'])

minmax_sse = []
for k in range(1, 12):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(minmax_scaled_df)
    minmax_sse.append(kmeans.inertia_)
plt.plot(range(1, 12), sse)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=7)
kmeans = kmeans.fit(minmax_scaled_df)
# Getting the cluster labels
predicted_clusters = kmeans.predict(minmax_scaled_df)

print(kmeans.predict([[0.67, 0.09]]))

print(predicted_clusters[0])
# Centroid values
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
y=minmax_scaled_df.iloc[:, 1].values
X=minmax_scaled_df.iloc[:, 0].values
plt.scatter(X, y, marker='.', c=predicted_clusters, s=50, cmap='viridis')
plt.show()

In [ ]:
# Repeat the process w/ robust scaling

In [ ]:
X = subset_df[['PCT_BLACK']]
y = subset_df[['Mean earnings of students working and not enrolled 10 years after entry']]
scaler = preprocessing.RobustScaler()
X_robust = scaler.fit(X)
y_robust = scaler.fit(y)
X_transformed = X_robust.transform(X)
y_tranformed = y_robust.transform(y)
plt.scatter(X_transformed, y_tranformed, marker='.')
plt.show()

In [ ]:
robust_scaled_df = scaler.fit_transform(subset_df)
robust_scaled_df = pd.DataFrame(robust_scaled_df, columns=['PCT', 'Earning'])

minmax_sse = []
for k in range(1, 12):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(robust_scaled_df)
    minmax_sse.append(kmeans.inertia_)
plt.plot(range(1, 12), sse)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=7)
kmeans = kmeans.fit(robust_scaled_df)
# Getting the cluster labels
predicted_clusters = kmeans.predict(robust_scaled_df)
# Centroid values
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
y=robust_scaled_df.iloc[:, 1].values
X=robust_scaled_df.iloc[:, 0].values
plt.scatter(X, y, marker='.', c=predicted_clusters, s=50, cmap='viridis')
plt.show()